## Project - Airline AI Assistant

In [ ]:
import json
import ollama
import gradio as gr


In [ ]:
MODEL = "llama3.2:latest"

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += " Give short, courteous answers, no more than 1 sentence."
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = ollama.chat(model=MODEL, messages=messages)
    return response.message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?



In [ ]:
# let's start by making a useful function
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city =  destination_city.lower()
    print(ticket_prices.get(city, "Unknown"))
    return ticket_prices.get(city, "Unknown")

In [ ]:
# There's a particular dictionary structure that's required to describe out function

price_function = {
    "name" : "get_ticket_price",
    "description" : "Get the price of a return ticket to the destination city. Call this whatever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city ",
    "parameters" :{
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that hte customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools
tools = [
    {"type": "function", "function": price_function}
]

## Getting OpenAI to use our Tool

The tool is included in the `tools` list, which is passed to the `ollama.chat` function as an argument.

The LLM will now be able to call our function whenever it needs to know the price of a ticket to a particular city.



In [ ]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_message}
    ] + history + [
        {"role": "user", "content": message}
    ]

    response = ollama.chat(model=MODEL, messages=messages, tools=tools)
    print(response)
    if response.get("done_reason") == "stop":
        tool_call_message = response["message"]
        tool_response, city = handle_tool_call(tool_call_message)
        messages.append(tool_call_message)
        messages.append(tool_response)
        response = ollama.chat(model=MODEL, messages=messages)
    return response.message.content

In [ ]:
## We have to write that function handle_tool_call

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = tool_call.function.arguments
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
    }
    return response, city

In [ ]:
# chat("How much is a ticket to Berlin", [])

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()